In [ ]:
# Install required packages
%pip install qiskit[visualization]
%pip install qiskit-aer
%pip install qiskit-ibm-runtime
%pip install qiskit-transpiler-service

In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import QFT
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_transpiler_service.transpiler_service import TranspilerService
from qkh2024.grader import scorer

In [ ]:
# Save your IBM Quantum account instance
# See here for more details: https://docs.quantum.ibm.com/start/setup-channel
QiskitRuntimeService.save_account(
    channel='ibm_quantum',
    instance='ibm-q-skku/quantum-hackathon-2024/hackathon',
    token='<MY_IBM_QUANTUM_TOKEN>',
    overwrite=True
)

In [ ]:
service = QiskitRuntimeService()
backend = service.backend("ibm_sherbrooke")
scorer = scorer()

## Qiskit Transpiler Service

In [ ]:
transpiler_list = []
transpiler_list.append(TranspilerService( 
    backend_name="ibm_sherbrooke", 
    ai="false", 
    optimization_level=1, 
))
transpiler_list.append(TranspilerService( 
    backend_name="ibm_sherbrooke", 
    ai="false", 
    optimization_level=2, 
))
transpiler_list.append(TranspilerService( 
    backend_name="ibm_sherbrooke", 
    ai="false", 
    optimization_level=3, 
))
transpiler_list.append(TranspilerService(
    backend_name="ibm_sherbrooke", 
    ai="true", 
    optimization_level=3, 
))

In [ ]:
tr_depths = [[], [], [], []]
tr_gate_counts = [[], [], [], []]
tr_cnot_counts = [[], [], [], []]
tr_scores = [[], [], [], []]

In [ ]:
# Transpile and collect result stats
num_qubits = np.arange(2, 15)

for nq in num_qubits:
    print(f'Start transpiling the {nq}-qubit circuit')
    circuit = QuantumCircuit(nq)
    circuit.h(range(nq))
    circuit.append(QFT(nq, do_swaps=False, inverse=True).decompose(), range(nq))
    for i in range(4):
        isa_circuit = transpiler_list[i].run(circuit)
        scorer.validate(circuit, isa_circuit, backend)
        tr_depths[i].append(isa_circuit.depth())
        tr_gate_counts[i].append(sum(isa_circuit.count_ops().values()))
        tr_cnot_counts[i].append(isa_circuit.num_nonlocal_gates())
        tr_scores[i].append(scorer.score(isa_circuit, backend))

In [ ]:
import matplotlib.pyplot as plt

ax = num_qubits
opt_list = ['Optimization Level 1', 'Optimization Level 2', 'Optimization Level 3', 'AI Optimization']
markers = ['o', 's', '^', '*']
linestyles = ['-', '--', '-.', ':']
colors = ['#FF6666', '#FFCC66', '#99FF99', '#66B2FF']

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10, 16))

# Plot 1: Circuit Depth
for i in range(4):
    ax1.plot(ax, tr_depths[i], label=opt_list[i], marker=markers[i], markersize=9, linestyle=linestyles[i], color=colors[i], linewidth=2)
ax1.set_xlabel("Number of qubits", fontsize=12)
ax1.set_ylabel("Depth", fontsize=12)
ax1.set_title("Circuit Depth of Transpiled Circuit", fontsize=14)
ax1.legend(fontsize=10)

# Plot 2: Total Number of Gates
for i in range(4):
    ax2.plot(ax, tr_gate_counts[i], label=opt_list[i], marker=markers[i], markersize=9, linestyle=linestyles[i], color=colors[i], linewidth=2)
ax2.set_xlabel("Number of qubits", fontsize=12)
ax2.set_ylabel("# of Total Gates", fontsize=12)
ax2.set_title("Total Number of Gates of Transpiled Circuit", fontsize=14)
ax2.legend(fontsize=10)

# Plot 3: Total Number of CNOTs
for i in range(4):
    ax3.plot(ax, tr_cnot_counts[i], label=opt_list[i], marker=markers[i],markersize=9, linestyle=linestyles[i], color=colors[i], linewidth=2)
ax3.set_xlabel("Number of qubits", fontsize=12)
ax3.set_ylabel("# of CNOTs", fontsize=12)
ax3.set_title("Total Number of CNOTs of Transpiled Circuit", fontsize=14)
ax3.legend(fontsize=10)

# Plot 4: Score of Transpiled Circuit
for i in range(4):
    ax4.plot(ax, tr_scores[i], label=opt_list[i], marker=markers[i], markersize=9, linestyle=linestyles[i], color=colors[i], linewidth=2)
ax4.set_xlabel("Number of qubits", fontsize=12)
ax4.set_ylabel("Score of Transpiled Circuit", fontsize=12)
ax4.set_title("Score of Transpiled Circuit", fontsize=14)
ax4.legend(fontsize=10)

fig.tight_layout()
plt.show()